<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

This file will become your README and also the index of your documentation.

## Install

```sh
pip install cylinder
```

This basic model provides the core function for a step change calculation in a mixed hot water cylinder

## How to use

Load some data that can be used to test the model - flow and electricity pricing

In [ ]:
df = (pd.DataFrame(load_demand(bed=3,unit=3)))
df.columns=["flow"]
df = df.merge(load_power(), how='left', left_index=True, right_index=True)
df.head()

,flow,price,price_kw,date,week,hour,day,peak,tou,cost,n_cost,sr_cost,lr_cost
timestamp,,,,,,,,,,,,,
2020-01-01 00:00:00,0.0,7.920,0.007920,2020-01-01,1,0,2,0,0.02,0.027920,-0.22080,0.162438,0.149272
2020-01-01 00:01:00,0.0,8.526,0.008526,2020-01-01,1,0,2,0,0.02,0.028526,-0.21474,0.158675,0.145694
2020-01-01 00:02:00,0.0,9.132,0.009132,2020-01-01,1,0,2,0,0.02,0.029132,-0.20868,0.155010,0.142235
2020-01-01 00:03:00,0.0,9.738,0.009738,2020-01-01,1,0,2,0,0.02,0.029738,-0.20262,0.151437,0.138888
2020-01-01 00:04:00,0.0,10.344,0.010344,2020-01-01,1,0,2,0,0.02,0.030344,-0.19656,0.147952,0.135645


Create a hot water cylinder object and initialise it with the data 

In [ ]:
hwc = HWC(T_set=68, T_deadband=2, element=3000, radius=.25, height=1)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/codespace/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_30580/2404404826.py", line 1, in <module>
    hwc = HWC(T_set=68, T_deadband=2, element=3000, radius=.25, height=1)
  File "/workspaces/cylinder/cylinder/hwc.py", line 48, in __init__
AttributeError: 'HWC' object has no attribute 'reset'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/codespace/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2052, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/codespace/.local/lib/python3.10/site-packages/IPython/core/ultratb.py", line 1112, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/home/codespace/.local/lib/python3.10/site-packages/IPython/core/ultratb.py", l

In [ ]:
hwc = HWC(T_set=68, T_deadband=2, element=3000, radius=.25, height=1)
# set the start temperature to the set temperature with a small random variation
hwc.T = hwc.T_set + np.random.uniform(-4, 0)
hwc.thermostat = 0
thermogram = np.ones([7,24])*(.5+3*.1)

results = []

for index, row in df[:500].iterrows():
  raw_flow = row['flow']
  hwc.flow = raw_flow*(hwc.T_demand-hwc.T_cold)/(hwc.T-hwc.T_cold)
  hwc._thermostat()
  hwc.T = hwc._update_temperatures(action=1)
  results.append([index,hwc.T, hwc.thermostat, hwc.flow,row.cost])
  r,c = row.day, row.hour
  thermogram[r,c] = hwc.thermostat * 1 * hwc.Q /60 + thermogram[r,c]*(1- 0.1)
  # print(hwc.T, hwc.thermostat)
  # hwc.T = hwc._update_temperatures(action = row['power'])
  # df.loc[index,'T'] = hwc.T
results = pd.DataFrame(results, columns=['time','T','thermostat','flow','cost']).set_index('time')

In [ ]:
hwc = HWC(T_set=68, T_deadband=2, element=3000, radius=.25, height=1)
# set the start temperature to the set temperature with a small random variation
hwc.T = hwc.T_set + np.random.uniform(-4, 0)
hwc.thermostat = 0
thermogram = np.ones([7,24])*(.5+3*.1)

results = []

for index, row in df[:500].iterrows():
  raw_flow = row['flow']
  hwc.flow = raw_flow*(hwc.T_demand-hwc.T_cold)/(hwc.T-hwc.T_cold)
  hwc._thermostat()
  hwc.T = hwc._update_temperatures(action=1)
  results.append([index,hwc.T, hwc.thermostat, hwc.flow,row.cost])
  r,c = row.day, row.hour
  thermogram[r,c] = hwc.thermostat * 1 * hwc.Q /60 + thermogram[r,c]*(1- 0.1)
  # print(hwc.T, hwc.thermostat)
  # hwc.T = hwc._update_temperatures(action = row['power'])
  # df.loc[index,'T'] = hwc.T
results = pd.DataFrame(results, columns=['time','T','thermostat','flow','cost']).set_index('time')